In [1]:
import numpy as np
import pandas as pd

In [2]:
tms_acutal_dir = "../../data/actual/TMS_Actual.csv"
flow_acutal_dir = "../../data/actual/FLOW_Actual.csv"
aws368_dir = "../../data/actual/AWS_368.csv"
aws541_dir = "../../data/actual/AWS_541.csv"
aws569_dir = "../../data/actual/AWS_569.csv"

In [3]:
tms = pd.read_csv(tms_acutal_dir)
flow = pd.read_csv(flow_acutal_dir)
aws368 = pd.read_csv(aws368_dir)
aws541 = pd.read_csv(aws541_dir)
aws569 = pd.read_csv(aws569_dir)

In [4]:
tms['SYS_TIME'] = pd.to_datetime(tms['SYS_TIME'])
flow['SYS_TIME'] = pd.to_datetime(flow['SYS_TIME'])
aws368['datetime'] = pd.to_datetime(aws368['datetime'])
aws541['datetime'] = pd.to_datetime(aws541['datetime'])
aws569['datetime'] = pd.to_datetime(aws569['datetime'])

In [5]:
tms = tms.set_index('SYS_TIME')
flow = flow.set_index('SYS_TIME')
aws368 = aws368.set_index('datetime')
aws541 = aws541.set_index('datetime')
aws569 = aws569.set_index('datetime')

In [65]:
aws569.info()

<class 'pandas.DataFrame'>
DatetimeIndex: 780790 entries, 2024-08-01 00:00:00 to 2026-01-25 05:09:00
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   YYMMDDHHMI  780790 non-null  float64
 1   STN         780789 non-null  float64
 2   TA          780698 non-null  float64
 3   RN_15m      775157 non-null  float64
 4   RN_60m      774756 non-null  float64
 5   RN_12H      769995 non-null  float64
 6   RN_DAY      775273 non-null  float64
 7   HM          780702 non-null  float64
 8   TD          780698 non-null  float64
dtypes: float64(9)
memory usage: 59.6 MB


In [6]:
# 컬럼명 변경
aws368 = aws368.add_suffix('_368')
aws541 = aws541.add_suffix('_541')
aws569 = aws569.add_suffix('_569')

# join
flow_df = flow.join([aws368, aws541, aws569], how='left')
tms_df = tms.join([aws368, aws541, aws569], how='left')


In [7]:
# 특정 패턴 포함 컬럼 삭제
cols_to_drop = [col for col in flow_df.columns if col in ['data_save_dt', 'YYMMDDHHMI_368', 'YYMMDDHHMI_541', 'YYMMDDHHMI_569', 'STN_368', 'STN_541', 'STN_569']]
flow_df = flow_df.drop(columns=cols_to_drop)

# 특정 패턴 포함 컬럼 삭제
cols_to_drop = [col for col in tms_df.columns if col in ['data_save_dt', 'YYMMDDHHMI_368', 'YYMMDDHHMI_541', 'YYMMDDHHMI_569', 'STN_368', 'STN_541', 'STN_569']]
tms_df = tms_df.drop(columns=cols_to_drop)


In [10]:
flow_df.head()

,flow_TankA,flow_TankB,level_TankA,level_TankB,TA_368,RN_15m_368,RN_60m_368,RN_12H_368,RN_DAY_368,HM_368,...,RN_DAY_541,HM_541,TD_541,TA_569,RN_15m_569,RN_60m_569,RN_12H_569,RN_DAY_569,HM_569,TD_569
SYS_TIME,,,,,,,,,,,,,,,,,,,,,
2025-09-02 23:53:00,230.4000,230.2500,3.55225,3.54900,23.5,0.0,0.0,0.0,0.0,83.9,...,0.0,92.7,21.6,22.9,NaN,NaN,NaN,NaN,86.7,20.6
2025-09-02 23:54:00,229.9875,230.1000,3.54700,3.54575,23.5,0.0,0.0,0.0,0.0,84.0,...,0.0,92.6,21.5,22.9,NaN,NaN,NaN,NaN,87.2,20.7
2025-09-02 23:55:00,229.5000,230.0625,3.54825,3.54475,23.5,0.0,0.0,0.0,0.0,84.1,...,0.0,92.6,21.5,22.9,NaN,NaN,NaN,NaN,87.3,20.7
2025-09-02 23:56:00,229.8375,230.4750,3.54125,3.54250,23.5,0.0,0.0,0.0,0.0,84.1,...,0.0,92.6,21.5,22.9,NaN,NaN,NaN,NaN,86.8,20.6
2025-09-02 23:57:00,229.9125,230.0250,3.54050,3.54050,23.5,0.0,0.0,0.0,0.0,84.1,...,0.0,92.5,21.5,23.0,NaN,NaN,NaN,NaN,86.9,20.7


In [11]:
tms_df.head()

,TOC_VU,PH_VU,SS_VU,FLUX_VU,TN_VU,TP_VU,TA_368,RN_15m_368,RN_60m_368,RN_12H_368,...,RN_DAY_541,HM_541,TD_541,TA_569,RN_15m_569,RN_60m_569,RN_12H_569,RN_DAY_569,HM_569,TD_569
SYS_TIME,,,,,,,,,,,,,,,,,,,,,
2024-08-26 15:09:00,4.1,7.2,0.9,3912.0,11.923,0.039,30.7,0.0,0.0,0.0,...,0.0,65.8,24.0,30.6,0.0,0.0,0.0,0.0,67.1,23.8
2024-08-26 15:10:00,4.1,7.2,0.8,3917.0,11.923,0.039,30.7,0.0,0.0,0.0,...,0.0,66.0,23.9,30.6,0.0,0.0,0.0,0.0,67.7,24.0
2024-08-26 15:11:00,4.1,7.1,0.9,3923.0,11.923,0.039,30.7,0.0,0.0,0.0,...,0.0,65.6,23.8,30.6,0.0,0.0,0.0,0.0,67.9,24.0
2024-08-26 15:12:00,4.1,7.1,0.9,3927.0,11.923,0.039,30.7,0.0,0.0,0.0,...,0.0,66.8,24.1,30.6,0.0,0.0,0.0,0.0,69.5,24.4
2024-08-26 15:13:00,4.1,7.1,2.2,3932.0,11.923,0.039,30.7,0.0,0.0,0.0,...,0.0,66.2,24.1,30.5,0.0,0.0,0.0,0.0,68.1,24.0


In [12]:
flow_df.info()

<class 'pandas.DataFrame'>
DatetimeIndex: 130732 entries, 2025-09-02 23:53:00 to 2025-12-03 10:39:00
Data columns (total 25 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   flow_TankA   130732 non-null  float64
 1   flow_TankB   130732 non-null  float64
 2   level_TankA  130732 non-null  float64
 3   level_TankB  130732 non-null  float64
 4   TA_368       129057 non-null  float64
 5   RN_15m_368   128431 non-null  float64
 6   RN_60m_368   128384 non-null  float64
 7   RN_12H_368   127450 non-null  float64
 8   RN_DAY_368   128458 non-null  float64
 9   HM_368       129057 non-null  float64
 10  TD_368       129057 non-null  float64
 11  TA_541       121846 non-null  float64
 12  RN_15m_541   117484 non-null  float64
 13  RN_60m_541   117293 non-null  float64
 14  RN_12H_541   113614 non-null  float64
 15  RN_DAY_541   117595 non-null  float64
 16  HM_541       122237 non-null  float64
 17  TD_541       121846 non-null  float64
 18  T

In [13]:
tms_df.info()

<class 'pandas.DataFrame'>
DatetimeIndex: 514551 entries, 2024-08-26 15:09:00 to 2025-09-29 05:23:00
Data columns (total 27 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   TOC_VU      426520 non-null  float64
 1   PH_VU       426520 non-null  float64
 2   SS_VU       426520 non-null  float64
 3   FLUX_VU     426525 non-null  float64
 4   TN_VU       426524 non-null  float64
 5   TP_VU       426520 non-null  float64
 6   TA_368      508956 non-null  float64
 7   RN_15m_368  508684 non-null  float64
 8   RN_60m_368  508561 non-null  float64
 9   RN_12H_368  505929 non-null  float64
 10  RN_DAY_368  508717 non-null  float64
 11  HM_368      508956 non-null  float64
 12  TD_368      508956 non-null  float64
 13  TA_541      514523 non-null  float64
 14  RN_15m_541  511212 non-null  float64
 15  RN_60m_541  510996 non-null  float64
 16  RN_12H_541  507836 non-null  float64
 17  RN_DAY_541  511270 non-null  float64
 18  HM_541      514523 no

In [14]:
# 1. 결측치 패턴 먼저 분석
def analyze_missing(df):
    # 연속된 결측치 길이 확인
    for col in df.columns:
        missing = df[col].isna()
        if missing.any():
            # 연속된 결측치 그룹 찾기
            groups = missing.ne(missing.shift()).cumsum()[missing]
            consecutive_counts = groups.value_counts().sort_index()
            
            print(f"\n{col}:")
            print(f"  총 결측: {missing.sum()}개")
            print(f"  최대 연속 결측: {consecutive_counts.max()}개")
            print(f"  결측 구간 수: {len(consecutive_counts)}개")


In [ ]:
# 보간
def conditional_interpolate(df):
    """
    max_gap: 보간할 최대 연속 결측 개수 (분)
    """
    df_filled = df.copy()
    
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        df_filled[col] = df[col].interpolate(method='time')
    
    return df_filled


In [20]:
analyze_missing(flow_df)
flow_df = conditional_interpolate(flow_df)


TA_368:
  총 결측: 300개
  최대 연속 결측: 300개
  결측 구간 수: 1개

RN_15m_368:
  총 결측: 912개
  최대 연속 결측: 606개
  결측 구간 수: 3개

RN_60m_368:
  총 결측: 955개
  최대 연속 결측: 650개
  결측 구간 수: 3개

RN_12H_368:
  총 결측: 1897개
  최대 연속 결측: 1303개
  결측 구간 수: 3개

RN_DAY_368:
  총 결측: 894개
  최대 연속 결측: 594개
  결측 구간 수: 2개

HM_368:
  총 결측: 300개
  최대 연속 결측: 300개
  결측 구간 수: 1개

TD_368:
  총 결측: 300개
  최대 연속 결측: 300개
  결측 구간 수: 1개

TA_541:
  총 결측: 8855개
  최대 연속 결측: 8026개
  결측 구간 수: 4개

RN_15m_541:
  총 결측: 13148개
  최대 연속 결측: 7636개
  결측 구간 수: 10개

RN_60m_541:
  총 결측: 13373개
  최대 연속 결측: 7636개
  결측 구간 수: 11개

RN_12H_541:
  총 결측: 17054개
  최대 연속 결측: 7636개
  결측 구간 수: 11개

RN_DAY_541:
  총 결측: 13083개
  최대 연속 결측: 7636개
  결측 구간 수: 9개

HM_541:
  총 결측: 8465개
  최대 연속 결측: 7636개
  결측 구간 수: 4개

TD_541:
  총 결측: 8855개
  최대 연속 결측: 8026개
  결측 구간 수: 4개

RN_15m_569:
  총 결측: 1757개
  최대 연속 결측: 1097개
  결측 구간 수: 3개

RN_60m_569:
  총 결측: 1893개
  최대 연속 결측: 1141개
  결측 구간 수: 4개

RN_12H_569:
  총 결측: 3855개
  최대 연속 결측: 1793개
  결측 구간 수: 3개

RN_DAY_569:
  총 결측: 1721개

In [21]:
analyze_missing(tms_df)
tms_df = conditional_interpolate(tms_df)


TOC_VU:
  총 결측: 83400개
  최대 연속 결측: 29123개
  결측 구간 수: 176개

PH_VU:
  총 결측: 83400개
  최대 연속 결측: 29123개
  결측 구간 수: 176개

SS_VU:
  총 결측: 83400개
  최대 연속 결측: 29123개
  결측 구간 수: 176개

FLUX_VU:
  총 결측: 83395개
  최대 연속 결측: 29118개
  결측 구간 수: 176개

TN_VU:
  총 결측: 83396개
  최대 연속 결측: 29119개
  결측 구간 수: 176개

TP_VU:
  총 결측: 83400개
  최대 연속 결측: 29123개
  결측 구간 수: 176개

TA_368:
  총 결측: 426개
  최대 연속 결측: 301개
  결측 구간 수: 4개

RN_15m_368:
  총 결측: 714개
  최대 연속 결측: 301개
  결측 구간 수: 7개

RN_60m_368:
  총 결측: 848개
  최대 연속 결측: 301개
  결측 구간 수: 7개

RN_12H_368:
  총 결측: 3632개
  최대 연속 결측: 824개
  결측 구간 수: 8개

RN_DAY_368:
  총 결측: 678개
  최대 연속 결측: 301개
  결측 구간 수: 7개

HM_368:
  총 결측: 426개
  최대 연속 결측: 301개
  결측 구간 수: 4개

TD_368:
  총 결측: 426개
  최대 연속 결측: 301개
  결측 구간 수: 4개

RN_15m_541:
  총 결측: 3286개
  최대 연속 결측: 930개
  결측 구간 수: 5개

RN_60m_541:
  총 결측: 3502개
  최대 연속 결측: 975개
  결측 구간 수: 5개

RN_12H_541:
  총 결측: 6662개
  최대 연속 결측: 1635개
  결측 구간 수: 5개

RN_DAY_541:
  총 결측: 3228개
  최대 연속 결측: 918개
  결측 구간 수: 5개

TA_569:
  총 결측: 54개
  최대 연속

In [9]:
flow_df.to_csv('../../data/processed/flow_processed.csv', date_format='%Y-%m-%d %H:%M:%S', index=True)
tms_df.to_csv('../../data/processed/tms_processed.csv', date_format='%Y-%m-%d %H:%M:%S', index=True)

In [22]:
flow_df.to_csv('../../data/processed/flow_processed_interpolation.csv', date_format='%Y-%m-%d %H:%M:%S', index=True)
tms_df.to_csv('../../data/processed/tms_processed_interpolation.csv', date_format='%Y-%m-%d %H:%M:%S', index=True)

In [17]:
test = pd.read_csv('../../data/processed/flow_processed.csv', 
                   parse_dates=[0], 
                   index_col=0)
print(test.head())

                     flow_TankA  flow_TankB  level_TankA  level_TankB  TA_368  \
SYS_TIME                                                                        
2025-09-02 23:53:00    230.4000    230.2500      3.55225      3.54900    23.5   
2025-09-02 23:54:00    229.9875    230.1000      3.54700      3.54575    23.5   
2025-09-02 23:55:00    229.5000    230.0625      3.54825      3.54475    23.5   
2025-09-02 23:56:00    229.8375    230.4750      3.54125      3.54250    23.5   
2025-09-02 23:57:00    229.9125    230.0250      3.54050      3.54050    23.5   

                     RN_15m_368  RN_60m_368  RN_12H_368  RN_DAY_368  HM_368  \
SYS_TIME                                                                      
2025-09-02 23:53:00         0.0         0.0         0.0         0.0    83.9   
2025-09-02 23:54:00         0.0         0.0         0.0         0.0    84.0   
2025-09-02 23:55:00         0.0         0.0         0.0         0.0    84.1   
2025-09-02 23:56:00         0.0      